In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, LSTM, Dense

def quantile_loss(q, y_true, y_pred):
    error = y_true - y_pred
    return tf.reduce_mean(tf.maximum(q * error, (q - 1) * error), axis=-1)

# Leer el archivo CSV comprimido
data = pd.read_csv('https://github.com/watorres/ProyectoDeepLearning/raw/main/device_gnss.csv.zip', compression='zip')

# Seleccionar las columnas relevantes para el análisis
selected_columns = ['TimeNanos', 'SvPositionXEcefMeters', 'SvPositionYEcefMeters', 'SvPositionZEcefMeters']
data = data[selected_columns]

# Eliminar filas con valores faltantes
data = data.dropna()

# Dividir los datos en características (X) y etiquetas (y)
X = data[['TimeNanos', 'SvPositionXEcefMeters', 'SvPositionYEcefMeters']]
y = data['SvPositionZEcefMeters']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Escalar los datos de entrada
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Reshape de los datos de entrada para ser compatibles con la red neuronal recurrente
X_train_reshaped = np.reshape(X_train_scaled, (X_train_scaled.shape[0], 1, X_train_scaled.shape[1]))
X_test_reshaped = np.reshape(X_test_scaled, (X_test_scaled.shape[0], 1, X_test_scaled.shape[1]))

# Definir el modelo de red neuronal recurrente bidireccional
model = Sequential()
model.add(Bidirectional(LSTM(64, return_sequences=True), input_shape=(1, X_train_scaled.shape[1])))
model.add(Bidirectional(LSTM(64)))
model.add(Dense(1))

# Compilar el modelo con la función de pérdida de error de percentil personalizada
model.compile(loss=lambda y_true, y_pred: quantile_loss(0.5, y_true, y_pred), optimizer='adam')

# Entrenar el modelo
model.fit(X_train_reshaped, y_train, epochs=10, batch_size=32)

# Evaluar el modelo en el conjunto de prueba
loss = model.evaluate(X_test_reshaped, y_test)
print(f'Loss en el conjunto de prueba: {loss}')

# Obtener las predicciones del modelo en el conjunto de prueba
y_pred = model.predict(X_test_reshaped)

# Calcular el error del percentil 50 y 95
error_percentil_50 = np.percentile(np.abs(y_pred.ravel() - y_test), 50)
error_percentil_95 = np.percentile(np.abs(y_pred.ravel() - y_test), 95)

print(f'Error del percentil 50: {error_percentil_50}')
print(f'Error del percentil 95: {error_percentil_95}')

Epoch 1/10
1341/1341 [==============================] - 24s 7ms/step - loss: 7137662.5000
Epoch 2/10
1341/1341 [==============================] - 9s 7ms/step - loss: 7137592.5000
Epoch 3/10
1341/1341 [==============================] - 9s 7ms/step - loss: 7137530.5000
Epoch 4/10
1341/1341 [==============================] - 9s 7ms/step - loss: 7137469.5000
Epoch 5/10
1341/1341 [==============================] - 10s 7ms/step - loss: 7137410.0000
Epoch 6/10
1341/1341 [==============================] - 10s 7ms/step - loss: 7137348.5000
Epoch 7/10
1341/1341 [==============================] - 8s 6ms/step - loss: 7137284.5000
Epoch 8/10
1341/1341 [==============================] - 9s 7ms/step - loss: 7137225.0000
Epoch 9/10
1341/1341 [==============================] - 9s 7ms/step - loss: 7137159.5000
Epoch 10/10
336/336 [==============================] - 2s 2ms/step - loss: 7140731.0000
Loss en el conjunto de prueba: 7140731.0
336/336 [==============================] - 2s 2ms/step
Error del pe

En este caso, el modelo parece tener una función de pérdida relativamente alta, en el rango de millones. Esto significa que el modelo aún no ha alcanzado un buen ajuste a los datos y puede requerir más entrenamiento o ajustes en la arquitectura/modelo.

El error del percentil 50 y 95 muestra el rango de errores en las predicciones del modelo. El error del percentil 50 es el valor en el medio, mientras que el error del percentil 95 es el valor en el percentil 95, lo que significa que el 95% de los errores se encuentran por debajo de ese valor.